In [206]:
import pandas as pd
pd.options.display.max_rows=100
df_out=pd.read_csv("heart.csv")
df_out.isnull().sum()
df_out.head()
df_out["Zscore"]=(df_out.Cholesterol-df_out["Cholesterol"].mean())/df_out["Cholesterol"].std()
df_out.head()
df_out=df_out[(df_out["Zscore"]<3) | (df_out["Zscore"]>-3)]
df_out


,Age,Sex,ChestPainType,RestingBP,Cholesterol,FastingBS,RestingECG,MaxHR,ExerciseAngina,Oldpeak,ST_Slope,HeartDisease,Zscore
0,40,M,ATA,140,289,0,Normal,172,N,0.0,Up,0,0.824621
1,49,F,NAP,160,180,0,Normal,156,N,1.0,Flat,1,-0.171867
2,37,M,ATA,130,283,0,ST,98,N,0.0,Up,0,0.769768
3,48,F,ASY,138,214,0,Normal,108,Y,1.5,Flat,1,0.138964
4,54,M,NAP,150,195,0,Normal,122,N,0.0,Up,0,-0.034736
...,...,...,...,...,...,...,...,...,...,...,...,...,...
913,45,M,TA,110,264,0,Normal,132,N,1.2,Flat,1,0.596068
914,68,M,ASY,144,193,1,Normal,141,N,3.4,Flat,1,-0.053020
915,57,M,ASY,130,131,0,Normal,115,Y,1.2,Flat,1,-0.619830
916,57,F,ATA,130,236,0,LVH,174,N,0.0,Flat,1,0.340090


In [207]:
from sklearn.preprocessing import LabelEncoder,OneHotEncoder
encoder=LabelEncoder()
df_out["Sex"]=encoder.fit_transform(df_out.Sex)
df_out["ExerciseAngina"]=encoder.fit_transform(df_out.ExerciseAngina)
# df_out["ChestPainType"]=encoder.fit_transform(df_out.ChestPainType)
# df_out["RestingECG"]=encoder.fit_transform(df_out.RestingECG)
encoder1=OneHotEncoder(sparse_output=False)
ct=["ChestPainType","RestingECG","ST_Slope"]
df1=encoder1.fit_transform(df_out[ct])
df2=pd.DataFrame(df1,columns=encoder1.get_feature_names_out(ct))
df_out.drop(ct,axis=1,inplace=True)
df3=pd.concat([df_out,df2],axis=1)
# df_out.drop(["ChestPainType","ST_Slope","RestingECG"],axis=1)
# df_out
df_out
df1
df2
df3.columns





Index(['Age', 'Sex', 'RestingBP', 'Cholesterol', 'FastingBS', 'MaxHR',
       'ExerciseAngina', 'Oldpeak', 'HeartDisease', 'Zscore',
       'ChestPainType_ASY', 'ChestPainType_ATA', 'ChestPainType_NAP',
       'ChestPainType_TA', 'RestingECG_LVH', 'RestingECG_Normal',
       'RestingECG_ST', 'ST_Slope_Down', 'ST_Slope_Flat', 'ST_Slope_Up'],
      dtype='object')

In [208]:
# df3.drop("Zscore",axis=1,inplace=True)
x=df3.drop(["HeartDisease","ChestPainType_ASY","RestingECG_LVH","ST_Slope_Down"],axis=1)
y=df3["HeartDisease"]
x.shape

(918, 16)

In [209]:
from sklearn.preprocessing import StandardScaler
scaler=StandardScaler()
arr=scaler.fit_transform(x)
arr

array([[-1.4331398 ,  0.51595242,  0.41090889, ..., -0.49044933,
        -1.00218103,  1.15067399],
       [-0.47848359, -1.93816322,  1.49175234, ..., -0.49044933,
         0.99782372, -0.86905588],
       [-1.75135854,  0.51595242, -0.12951283, ...,  2.03894663,
        -1.00218103,  1.15067399],
       ...,
       [ 0.37009972,  0.51595242, -0.12951283, ..., -0.49044933,
         0.99782372, -0.86905588],
       [ 0.37009972, -1.93816322, -0.12951283, ..., -0.49044933,
         0.99782372, -0.86905588],
       [-1.64528563,  0.51595242,  0.30282455, ..., -0.49044933,
        -1.00218103,  1.15067399]])

In [210]:
from sklearn.model_selection import train_test_split
x_train,y_train,x_val,y_val=train_test_split(arr,y,test_size=0.2,random_state=42)

In [215]:
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
model=LogisticRegression()
model.fit(x_train,x_val)
y_predict=model.predict(y_train)
model1=SVC()
model1.fit(x_train,x_val)
y1_predict=model1.predict(y_train)
y_predict

model2=RandomForestClassifier()
model2.fit(x_train,x_val)
y2_predict=model2.predict(y_train)
print(model2.score(x_train,x_val))




1.0


In [198]:
from sklearn.metrics import accuracy_score
# result=accuracy_score(y_val,y_predict)
# result1=accuracy_score(y_val,y1_predict)
# print(result)
print(model.score(x_train,x_val))
# result1
print(model1.score(x_train,x_val))

0.8719346049046321
0.9100817438692098


In [199]:
from sklearn.decomposition import PCA
pca=PCA(0.95)
x_pca=pca.fit_transform(x)
pca.explained_variance_ratio_
# pca.n_components_
print(x_pca)

[[ 92.3149887   29.44568058]
 [-17.1445248   13.73603475]
 [ 81.91034202 -38.22349875]
 ...
 [-69.00655181 -17.3346223 ]
 [ 39.20822335  33.59196558]
 [-21.43866668  37.21661282]]


In [200]:
x_pca
x_train,y_train,x_val,y_val=train_test_split(x_pca,y,test_size=0.2,random_state=42)
print(x_train)

[[  44.31023378   57.34448909]
 [  12.66795026   42.3896619 ]
 [  13.77321018  -16.54589119]
 ...
 [  54.42058574    3.4783204 ]
 [-199.2441824   -13.11422991]
 [ 192.87783616  -18.51539034]]


In [204]:
models={
    "LogisticRegression":LogisticRegression(max_iter=1000),
    "SVC":SVC(),
    "randomforest":RandomForestClassifier()
}
for model in models.values():
    # print(x_train)
    model.fit(x_train,x_val)
    model.predict(y_train)
    print(model.score(x_train,x_val))
  

0.6989100817438693
0.715258855585831
1.0


In [203]:
model1=SVC()
model1.fit(x_train,x_val)
y_predict=model1.predict(y_train)
model1.score(x_train,x_val)


0.715258855585831